In [1]:
#!/usr/bin/env python
# coding: utf-8
from src.dynapee import PriceModel
from src.datastruct import *
from src.utils import *
from price_optimization_api import *
import numpy as np
import pandas as pd
import time
from datetime import datetime, timedelta
import abc

price_api = PriceAPI()
listing_id = "54338248"
calendar_date = "2023-10-19"
dummy = PropertyAttribute(100,38,5,4,5,2,0,0,1,0,'54338248')
share = price_api.compute_share(dummy,calendar_date)
share

C:\Data\QuibbleRM\jenkins\price-optimization\price_optimization_api.py:172: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  market_data["mc"] = market_data["calendarDate"].apply(get_mc_factor)


array([0.67423939, 0.15273917, 0.17302144])

In [3]:

#results = price_api.optimize_price(listing_id,calendar_date)

C:\Data\QuibbleRM\jenkins\price-optimization\price_optimization_api.py:173: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  market_data["mc"] = market_data["calendarDate"].apply(get_mc_factor)


In [4]:
results

,price,review_count,Adjusted,bedrooms,rating_value,minNights,dist,pool,jacuzzi,landscape_views,id,available,calendarDate,listing_hashId,mc,ToOptimize,Optimized_Price
0,0,38,4.984420,4,5.00,2,0,0,1,0,54338248,False,2023-10-19,63b8d8a3743bd2fd6c76658c,6.612567,1,432.216193
4,266,19,5.178765,4,4.47,1,0,0,0,1,51235365,True,2023-10-19,62b0d382b18efee00cd47374,6.612567,0,0.000000
5,308,6,4.477023,4,5.00,1,0,0,1,0,53172716,True,2023-10-19,62b0d629b18efee00cd477c4,6.612567,0,0.000000


[[100.          38.           5.           4.           5.
    2.           0.           0.           1.           0.        ]
 [266.          19.           5.17876514   4.           4.47
    1.           0.           0.           0.           1.        ]
 [308.           6.           4.47702301   4.           5.
    1.           0.           0.           1.           0.        ]]


C:\Data\QuibbleRM\jenkins\price-optimization\price_optimization_api.py:173: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  market_data["mc"] = market_data["calendarDate"].apply(get_mc_factor)


0

In [7]:
share

0

In [2]:
mc_factor = pd.read_csv("bookable_search.csv")
def get_mc_factor(calendar_date: str):
            
            
    date_obj = datetime.strptime(calendar_date, "%Y-%m-%d")
    day_of_week = date_obj.strftime("%a")
    month = date_obj.strftime("%B")
    q = f'Month == "{month}" & Day == "{day_of_week}"'
    factor = mc_factor.query(q)
            
    return factor.Bookable_Search.iloc[0]

In [3]:
def optimize_price(dat, choice = 1):
            
    m = dat.copy()
    m['price'] = m.price.astype(str)
    m['price'] = m['price'].str.replace('$', '')
    m['price'] = m['price'].str.replace(',', '')
    mat =  m.iloc[:,:10].values.astype(float)
    dynasaur = PriceModel(market_matrix = mat, coeff = [-0.0062, 0.0003, 0.0879, 0.1106, 0.3239, 0.015, 0.0002, 0.011, 0.42, 0.141], mc = choice)
    res = dynasaur.optimize()
    m["Optimized_Price"] = 0
    i = 0  
    j = "Optimized_Price"
    m.at[i, j] = res[1]
    
    return m

In [4]:
def get_availability_info(listing_ids: list[str], calendar_date: list[str], lag:int = 1):
    
    date_yesterday = (datetime.now() - timedelta(days=lag))
   
    start_of_day = date_yesterday.replace(hour=0, minute=0, second=0)
    end_of_day = date_yesterday.replace(hour=23, minute=59, second=59)

    availability_collection  = merlin_hunter["scrapy_quibble"]["scrapy_availability"]
    availability_match = {
            "listing_id": {"$in": listing_ids},
            "calendarDate": {"$in": calendar_date},
            "minNights": {"$lt": 30},
            "scraped_date": {
                    "$gte": start_of_day,
                    "$lt": end_of_day
                }

        }

  
    availability_query = [
                {
                    "$match": availability_match
                },
                {
                    "$project": {
                        "_id": 0,
                        "id": "$listing_id",
                        "calendarDate": "$calendarDate",
                        "scraped_date": "$scraped_date",
                        "available": "$available",
                        "minNights": "$minNights",
                        "price": { "$ifNull" : [ "$price", 0 ] }
                    }
                }

            ]
    
    availabilities: Iterable[dict] = availability_collection.aggregate(availability_query)

    available_list = []
    
    for _avail in availabilities:

        if not _avail.get('id'):
            continue

        available_list.append(_avail)
        
    return available_list

In [5]:
listing_id = "54338248"
calendar_date = "2023-10-19"
client_property_data = get_property_info([listing_id])[0]
rental_market = ClientProperty(id = client_property_data["listing_id"],competitors = client_property_data["intelCompSet"])

all_ids = []
all_ids.append(client_property_data["listing_id"])
all_ids.extend(client_property_data["intelCompSet"])
all_ids = list(set(all_ids))
print(all_ids)

['43141319', '617961959918601179', '51765332', '26000178', '51235365', '50661163', '53172716', '38027584', '54338248', '634847614835309408', '26427218']


In [6]:
image_set = get_image_scores(all_ids)
image_set = pd.DataFrame(image_set)
image_scores = image_set.groupby('Listings')['Score'].agg(list).reset_index()
image_scores.columns = ["id","Scores"]
image_scores["Values"] = image_scores.apply(odd_weighted_average, axis=1)
image_scores[['Reference', 'Adjusted', 'Factor']] = image_scores['Values'].apply(lambda x: pd.Series(x))
image_scores = image_scores[["id","Scores","Reference","Adjusted"]]

In [7]:
comp_list = []
[comp_list.append(x) for x in rental_market._competitors]


client_listing = pd.DataFrame(get_listing_info([listing_id]))
competitor_listing = pd.DataFrame(get_listing_info(comp_list))
market_listing = pd.concat([client_listing,competitor_listing],axis = 0)
market_listing["bedrooms"] = pd.to_numeric(market_listing["bedrooms"], errors="coerce").fillna(0).astype(int)
market_listing.rename(columns={'_id': 'listing_hashId'}, inplace=True)
market_listing = parse_scrap_info(market_listing)
market_listing = pd.merge(market_listing,image_scores, on = "id", how = "outer")
market_listing["Reference"].fillna(market_listing["Reference"].mean(),inplace = True)
market_listing["Adjusted"].fillna(market_listing["Adjusted"].mean(),inplace = True)


In [8]:
market_availabilities = pd.DataFrame(get_availability_info(all_ids,[calendar_date],1))


In [9]:
market_availabilities.calendarDate

0    2023-10-19
1    2023-10-19
2    2023-10-19
3    2023-10-19
4    2023-10-19
5    2023-10-19
6    2023-10-19
7    2023-10-19
Name: calendarDate, dtype: object

In [10]:
market_listing= pd.merge(market_listing,market_availabilities,on="id", how = 'inner')
market_listing['dist'] = 0

In [23]:
market_data = market_listing[["price","review_count","Adjusted","bedrooms","rating_value","minNights","dist","pool","jacuzzi","landscape_views","id","available","calendarDate","listing_hashId"]]


In [24]:
market_data["mc"] = market_data["calendarDate"].apply(get_mc_factor)

C:\Users\rosen\AppData\Local\Temp\ipykernel_17384\1779695634.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  market_data["mc"] = market_data["calendarDate"].apply(get_mc_factor)


In [25]:
market_data = market_data.drop_duplicates(subset = ['id'])

In [26]:
market_data["ToOptimize"] = market_data['id'].apply(lambda x: 1 if str(x) == str(rental_market._id) else 0)

In [27]:
market_data = market_data.query('available == True or ToOptimize == 1')

In [28]:
market_data = market_data.sort_values(by = 'ToOptimize', ascending = False)

In [31]:
market_data

,price,review_count,Adjusted,bedrooms,rating_value,minNights,dist,pool,jacuzzi,landscape_views,id,available,calendarDate,listing_hashId,mc,ToOptimize
0,0,38,4.984420,4,5.00,2,0,0,1,0,54338248,False,2023-10-19,63b8d8a3743bd2fd6c76658c,6.612567,1
4,$266,19,5.178765,4,4.47,1,0,0,0,1,51235365,True,2023-10-19,62b0d382b18efee00cd47374,6.612567,0
5,$308,6,4.477023,4,5.00,1,0,0,1,0,53172716,True,2023-10-19,62b0d629b18efee00cd477c4,6.612567,0


In [33]:
class PropertyAttribute:

    
    _price = 0
    _review_count = 0
    _image_score = 0
    _bedroom = 0
    _rating_value = 0
    _min_stay = 0
    _distance = 0
    _pool = 0
    _jacuzzi = 0
    _landscape_views = 0
    _id = 0

    def __init__(self,price,review_count,image_score,bedroom,rating_value,min_stay,distance,pool,jacuzzi,landscape_views,id):

        self._price = price
        self._review_count = review_count
        self._image_score = image_score
        self._bedroom = bedroom
        self._rating_value = rating_value
        self._min_stay = min_stay
        self._distance = distance
        self._pool = pool
        self._jacuzzi = jacuzzi
        self._landscape_views = landscape_views
        self._id = id

In [36]:
dummy = PropertyAttribute(100,38,5,4,5,2,0,0,1,0,'54338248')

In [37]:
def instance_to_array(instance):
    return [value for key, value in instance.__dict__.items()]

In [39]:
df = market_data.copy()

In [45]:
instance_to_array(dummy)

[100, 38, 5, 4, 5, 2, 0, 0, 1, 0, '54338248']

In [48]:
df.iloc[0, 0:11] = instance_to_array(dummy)

In [49]:
df

,price,review_count,Adjusted,bedrooms,rating_value,minNights,dist,pool,jacuzzi,landscape_views,id,available,calendarDate,listing_hashId,mc,ToOptimize
0,100,38,5.000000,4,5.00,2,0,0,1,0,54338248,False,2023-10-19,63b8d8a3743bd2fd6c76658c,6.612567,1
4,$266,19,5.178765,4,4.47,1,0,0,0,1,51235365,True,2023-10-19,62b0d382b18efee00cd47374,6.612567,0
5,$308,6,4.477023,4,5.00,1,0,0,1,0,53172716,True,2023-10-19,62b0d629b18efee00cd477c4,6.612567,0


In [25]:
to_optimize = (market_data['ToOptimize'] == 1).any()

In [26]:
to_optimize

True

In [27]:
num_comp = market_data.shape[0]

In [28]:
num_comp

3

In [32]:
i = float(market_data.iloc[0]["mc"])
optim = optimize_price(market_data,i)

In [33]:
optim

,price,review_count,Adjusted,bedrooms,rating_value,minNights,dist,pool,jacuzzi,landscape_views,available,id,calendarDate,listing_hashId,mc,ToOptimize,Optimized_Price
0,0,38,4.984420,4,5.00,2,0,0,1,0,False,54338248,2023-10-19,63b8d8a3743bd2fd6c76658c,6.612567,1,432.216193
4,266,19,5.178765,4,4.47,1,0,0,0,1,True,51235365,2023-10-19,62b0d382b18efee00cd47374,6.612567,0,0.000000
5,308,6,4.477023,4,5.00,1,0,0,1,0,True,53172716,2023-10-19,62b0d629b18efee00cd477c4,6.612567,0,0.000000
